In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загрузка данных
file_path = 'filtered_schedule_data.csv'
data = pd.read_csv(file_path)

# Очистим имена столбцов от пробелов
data.columns = data.columns.str.strip()

# Проверим, какие столбцы присутствуют в DataFrame
print("Столбцы в DataFrame:", data.columns)

# Преобразуем время в числовой формат (минуты с начала дня)
def time_to_minutes(time_str):
    """Преобразует строку времени HH:MM:SS AM/PM в количество минут с начала дня."""
    try:
        time_obj = pd.to_datetime(time_str, format='%I:%M:%S %p')
        return time_obj.hour * 60 + time_obj.minute
    except:
        return None  # Возвращаем None, если не удается преобразовать

# Преобразуем столбцы с временем
data['Время подхода судна в минутах от начала суток'] = data['Время подхода судна в минутах от начала суток'].apply(time_to_minutes)
data['Время отхода судна в минутах от начала суток'] = data['Время отхода судна в минутах от начала суток'].apply(time_to_minutes)

# Убедимся, что данные преобразованы корректно
print(data[['Время подхода судна в минутах от начала суток', 'Время отхода судна в минутах от начала суток']].head())

# Теперь определим признаки (X) и целевую переменную (y)
X = data[['Время подхода судна в минутах от начала суток', 
          'Время отхода судна в минутах от начала суток', 
          'Время стоянки', 
          'Причаливание/подход', 
          'Отход']]  # Замените на нужные столбцы
y = data['Время отхода судна в минутах от начала суток']  # Или другой столбец для целевой переменной

# Преобразуем категориальные данные в числовой формат, если они есть
X = pd.get_dummies(X, drop_first=True)  # Если нужно, заменим категориальные данные на дамми-переменные

# Стандартизация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Теперь можно продолжать обучение модели
print("Данные успешно подготовлены для обучения.")


Столбцы в DataFrame: Index(['Уникальный ID записи', 'ID причала', 'Название причала',
       'Название судна - отсутсвует если отменено', 'Название маршрута',
       'Швартовочное место', 'Дата начала действия записи расписания в UTC',
       'Дата окончания действиязаписи  расписания в UTC',
       'Время подхода судна в минутах от начала суток',
       'Время отхода судна в минутах от начала суток', 'Время стоянки',
       'Название судна которое было отменено',
       'Тип записи - пустое знеачение если изменений нет, Отмена - судно отменено',
       'Дата начала действия записи расписания',
       'Дата окончания действия записи расписания', 'Причаливание/подход',
       'Отход'],
      dtype='object')
   Время подхода судна в минутах от начала суток  \
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4                       

D:\pythonProject1\venv\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\pythonProject1\venv\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\pythonProject1\venv\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [10]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('filtered_schedule_data.csv')

# Заполнение пропусков в числовых столбцах
data['Время подхода судна в минутах от начала суток'] = data['Время подхода судна в минутах от начала суток'].fillna(0)
data['Время отхода судна в минутах от начала суток'] = data['Время отхода судна в минутах от начала суток'].fillna(0)

# Заполнение пропусков в строковых столбцах
data['Название судна - отсутсвует если отменено'] = data['Название судна - отсутсвует если отменено'].fillna('Не указано')
data['Название судна которое было отменено'] = data['Название судна которое было отменено'].fillna('Не указано')
data['Тип записи - пустое знеачение если изменений нет, Отмена - судно отменено'] = data['Тип записи - пустое знеачение если изменений нет, Отмена - судно отменено'].fillna('Нет изменений')

# Преобразование данных в числовой формат
data['Время подхода судна в минутах от начала суток'] = pd.to_numeric(data['Время подхода судна в минутах от начала суток'], errors='coerce')
data['Время отхода судна в минутах от начала суток'] = pd.to_numeric(data['Время отхода судна в минутах от начала суток'], errors='coerce')

# Проверим наличие пропусков после обработки
print(data.isna().sum())

# Теперь можно продолжить с разделением признаков и целевой переменной
X = data[['distance_km', 'hour', 'timetable.duration', 'time_of_day_morning', 'time_of_day_evening', 'time_of_day_night']]
y = data['trip_duration']

# Продолжаем нормализацию и обучение модели
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделим на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Обучаем модель случайного леса
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

# Оценка модели
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'MAE: {mae:.2f} минут')
print(f'RMSE: {rmse:.2f} минут')

# Сохраняем модель
import joblib
joblib.dump(model, 'trip_duration_model.pkl')


Уникальный ID записи                                                         0
ID причала                                                                   0
Название причала                                                             0
Название судна - отсутсвует если отменено                                    0
Название маршрута                                                            0
Швартовочное место                                                           0
Дата начала действия записи расписания в UTC                                 0
Дата окончания действиязаписи  расписания в UTC                              0
Время подхода судна в минутах от начала суток                                0
Время отхода судна в минутах от начала суток                                 0
Время стоянки                                                                0
Название судна которое было отменено                                         0
Тип записи - пустое знеачение если изменений нет, От

KeyError: "None of [Index(['distance_km', 'hour', 'timetable.duration', 'time_of_day_morning',\n       'time_of_day_evening', 'time_of_day_night'],\n      dtype='object')] are in the [columns]"

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

# Загрузка очищенных данных
data_filtered = pd.read_csv("filtered_schedule_data.csv")

# Преобразуем время подхода и отхода в числовой формат (если они не были ранее)
data_filtered['Время подхода судна в минутах от начала суток'] = pd.to_numeric(data_filtered['Время подхода судна в минутах от начала суток'], errors='coerce')
data_filtered['Время отхода судна в минутах от начала суток'] = pd.to_numeric(data_filtered['Время отхода судна в минутах от начала суток'], errors='coerce')

# Допустим, что целевая переменная - это время поездки (т.е. разница между временем подхода и отхода)
data_filtered['trip_duration'] = data_filtered['Время отхода судна в минутах от начала суток'] - data_filtered['Время подхода судна в минутах от начала суток']

# Убедимся, что нет пропущенных значений в целевой переменной
data_filtered = data_filtered.dropna(subset=['trip_duration'])

# Признаки (features) для обучения модели
X = data_filtered[['Время подхода судна в минутах от начала суток', 'Время отхода судна в минутах от начала суток']]

# Целевая переменная (target)
y = data_filtered['trip_duration']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных (это полезно для многих моделей)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание и обучение модели RandomForest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказания
y_pred = model.predict(X_test_scaled)

# Оценка качества модели
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Выводим результаты
print(f'Mean Absolute Error (MAE): {mae:.2f} minutes')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f} minutes')



Mean Absolute Error (MAE): 2.83 minutes
Root Mean Squared Error (RMSE): 15.05 minutes


D:\pythonProject1\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

# Загрузка очищенных данных
data_filtered = pd.read_csv("filtered_schedule_data.csv")

# Преобразуем время подхода и отхода в числовой формат (если они не были ранее)
data_filtered['Время подхода судна в минутах от начала суток'] = pd.to_numeric(data_filtered['Время подхода судна в минутах от начала суток'], errors='coerce')
data_filtered['Время отхода судна в минутах от начала суток'] = pd.to_numeric(data_filtered['Время отхода судна в минутах от начала суток'], errors='coerce')

# Допустим, что целевая переменная - это время поездки (т.е. разница между временем подхода и отхода)
data_filtered['trip_duration'] = data_filtered['Время отхода судна в минутах от начала суток'] - data_filtered['Время подхода судна в минутах от начала суток']

# Убедимся, что нет пропущенных значений в целевой переменной
data_filtered = data_filtered.dropna(subset=['trip_duration'])

# Признаки (features) для обучения модели
X = data_filtered[['Время подхода судна в минутах от начала суток', 'Время отхода судна в минутах от начала суток']]

# Целевая переменная (target)
y = data_filtered['trip_duration']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных (это полезно для многих моделей)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание и обучение модели RandomForest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказания
y_pred = model.predict(X_test_scaled)

# Оценка качества модели
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)  # Коэффициент детерминации (R^2)

# Выводим результаты
print(f'Mean Absolute Error (MAE): {mae:.2f} minutes')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f} minutes')
print(f'R-squared (R^2): {r2:.2f}')

# Сохранение модели в формате ONNX
# Преобразуем модель в формат ONNX
onnx_model = convert_sklearn(model, initial_types=[('float_input', FloatTensorType([None, X_train_scaled.shape[1]]))])

# Сохранение модели в файл
onnx.save_model(onnx_model, 'water_taxi_model.onnx')

print("Модель успешно сохранена в формате ONNX.")


D:\pythonProject1\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean Absolute Error (MAE): 2.83 minutes
Root Mean Squared Error (RMSE): 15.05 minutes
R-squared (R^2): 0.96
Модель успешно сохранена в формате ONNX.
